Olena's Important Columns in "StartInfo" CSVs
Notes:
 - please name videos based on what mice they are, don't include the name of the two mice that were in both chambers b/c that gets cumbersome to memorize which is which based on what camera they came from (Cam1 vs. Cam2), all names of files/(their directories leading to it) should be standalone

1. type of experiment columns (conditioning, ..., renewal)
2. camera_type
3. animal_id (will only need this info for video name)
4. chamber
5. experimental_group (important for grouping averages)

Patrick's Important Columns in "StartInfo" CSVs

1. type of experiment columns (conditioning, ..., renewal)
2. animal_id
3. chamber
4. experimental_group

Ozge's Important Columns in "StartInfo" CSVs

1. type of experiment columns (conditioning, ..., renewal)
2. animal_id
3. chamber
4. experimental_group

In [7]:
import cv2
import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
import holoviews as hv
import EzTrackFunctions as ez
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import FreezeAnalysis_Functions as fz

# Defining Functions

In [8]:
def find_file_with_strings(root_path, strings_list, dont_include_suffix):
    strings_list = [s.lower() for s in strings_list]
    print(strings_list)
    for root, dirs, files in os.walk(root_path):
        for file in files:
            # ignore files that start with a dot (annoying lock files)
            if file.startswith("."):
                continue
            else:
                file_path = os.path.join(root, file).lower()
                if all(string.lower() in file_path for string in strings_list) and dont_include_suffix.lower() not in file_path:
                    return os.path.join(root, file)
    return None

def find_file_with_strings(root_path, strings_list):
    strings_list = [s.lower() for s in strings_list]
    print(strings_list)
    for root, dirs, files in os.walk(root_path):
        for file in files:
            # ignore files that start with a dot (annoying lock files)
            if file.startswith("."):
                continue
            else:
                file_path = os.path.join(root, file).lower()
                if all(string.lower() in file_path for string in strings_list):
                    return os.path.join(root, file)
    return None

def find_directory_with_strings(strings, root_directory):
    # Convert strings to lowercase
    strings_lower = [s.lower() for s in strings]

    # Iterate over directories in root directory
    for dirpath, _, _ in os.walk(root_directory):
        # Convert directory path to lowercase
        dirpath_lower = dirpath.lower()
        
        # Check if all strings are in the directory path
        if all(s in dirpath_lower for s in strings_lower):
            return dirpath

    # If no directory is found, return None
    return None

def get_adjusted_fps(video_file_path, camera_type):
    # Open video file
    cap = cv2.VideoCapture(video_file_path)

    # Get FPS
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Adjust FPS if camera type is not 'usb'
    if camera_type != 'usb':
        fps /= 2

    # Release video capture object
    cap.release()

    # Return adjusted FPS
    return fps

def extract_mouse_id_olena(filename):
    match = re.search(r"(hSyn-AS-Gi-\d+)", filename)
    if match:
        return match.group(1)
    else:
        return None


def extract_mouse_id_ozge(filename):
    match = re.search(r"C\d+", filename)
    if match:
        return match.group()
    else:
        return None

def extract_mouse_id_patrick(filename):
    return filename.split("_")[0]

def remove_substring(original_string, substring):
    return original_string.replace(substring, "")

def time_to_frames(time, fps):
    time_str = str(time)
    if ":" in time_str:
        # Convert time in minutes to time in seconds
        minutes, seconds = map(int, time_str.split(":"))
        time_in_seconds = 60 * minutes + seconds
    else:
        time_in_seconds = int(time)

    # Calculate number of frames based on time and fps
    frames = int(time_in_seconds * fps)

    return frames

# Defining Variables

In [23]:
# this will determine some parameters going into the automated analysis
person = "Olena"
# "usb" or "videofreeze"
which_camera = "videofreeze"
# Adjust these parameters based on trial and error
FreezeThresh = 180 
MinDuration = 40
# this should be the suffix of your info sheet, if not, change it
info_sheet_suffix  = "_FC_startime_info.csv"
# this should be the suffix of your eztrack output, if not, change it
eztrack_output_processed_suffix = "FreezingOutput_processed.csv"
vid_file_extension = ".avi"
# your root path
PERSON_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group"
# your calibration root path, if the same as above, c&p the above
CALIBRATION_ROOT = f"/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/calibration/{which_camera}"
# how much time after the event occurred do you want to get? in seconds

# videofreeeze_variables files - it contains the FreezeThresh and MinDuration variables for each mouse - we will avg while running the program
videofreeze_variables = f"{PERSON_ROOT}/VideoFreeze_variables.csv"

half_time_window = 2.5
# most likely you don't need to change this
calibrate_video_what_frame_to_start = 0
number_of_frames_to_calibrate = 600
h,w = 300,1000
dsmpl = 1
vid_d_start = 0
# change if you want to track another event, make sure name matchs column name
event_tracked = 'CS ON'

In [10]:
# defining columns we will be looking at for each person
experiment_cols = {
        "Olena": ["conditioning", "extinction_1", "extinction_2", "retrieval", "late_retrieval", "renewal"],
        "Ozge": ["conditioning", "extinction_1"],
        "Patrick": ["conditioning", "extinction_1", "retrieval"],
    }

general_cols = {
    "Olena":["animal_id", "camera_type"],
    "Ozge": ["animal_id"],
    "Patrick": ["animal_id"],
}

# we also focus on : chamber and experimental_group columns but we don't define that here

In [11]:
start_info_filepath = os.path.join(PERSON_ROOT, f"{person.lower()}{info_sheet_suffix}")
print(f"start_info_filepath: {start_info_filepath}")

df_start_info = pd.read_csv(start_info_filepath)
df_start_info.head()

start_info_filepath: /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/olena_FC_startime_info.csv


,animal_name,animal_id,chamber,experimental_group,camera_id,camera_type,conditioning,extinction_1,extinction_2,retrieval,late_retrieval,renewal
0,C10208,hSyn-AS-Gi-1,A,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
1,C10208,hSyn-AS-Gi-1,A,AS-Gi,1,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
2,C10209,hSyn-AS-Gi-2,C,AS-mCherry,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
3,C10208,hSyn-AS-Gi-2,A,AS-Gi,2,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
4,C10210,hSyn-AS-Gi-3,B,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000


In [17]:
videofreeze_variables_df = pd.read_csv(videofreeze_variables)
videofreeze_variables_df.head()

,Video name,FreezeThresh,MinDuration,Chamber
0,080822_conditioning\VideoFreeze\hSyn-AS-Gi-1.avi,200,30,A
1,080822_conditioning\VideoFreeze\hSyn-AS-Gi-5.avi,245,15,A
2,080922_extinction_1\VideoFreeze\hSyn-AS-Gi-1.avi,200,30,NaN
3,080922_extinction_1\VideoFreeze\hSyn-AS-Gi-2.avi,235,10,NaN
4,081022_extinction_2\VideoFreeze\hSyn-AS-Gi-1.avi,175,40,NaN


## Pre-processing the videofreeze_variables_df to avg each experimental condition

In [20]:
experiment_names = []

for idx, col in videofreeze_variables_df.iterrows():
    # the object that i have in each iteration is a pandas series
    # i can access the values of the series by using the column names
    vid_path = col["Video name"]
    experiment_name = "_".join(vid_path.split("\\")[0].split("_")[1:])
    #print(experiment_name)
    experiment_names.append(experiment_name)

print(experiment_names)
videofreeze_variables_df["experiment_name"] = experiment_names
#videofreeze_variables_df.head()

videofreeze_variables_df_avgd = videofreeze_variables_df.groupby("experiment_name")["FreezeThresh", "MinDuration"].mean()
videofreeze_variables_df_avgd.head(6)

['conditioning', 'conditioning', 'extinction_1', 'extinction_1', 'extinction_2', 'extinction_2', 'retrieval', 'retrieval', 'late_retrieval', 'late_retrieval', 'renewal', 'renewal']


,FreezeThresh,MinDuration
experiment_name,,
conditioning,222.5,22.5
extinction_1,217.5,20.0
extinction_2,205.0,27.5
late_retrieval,200.0,30.0
renewal,200.0,30.0
retrieval,200.0,30.0


## The following code blocks can be run independently

# Calibration & FreezeAnalysis

In [ ]:
# going through experiment for this person
for experiment in experiment_cols[person]:
    print(experiment)
    # now go through each mouse for this person
    for index, mouse_row in df_start_info.iterrows():
        if mouse_row["camera_type"] == which_camera:
            
            if mouse_row["camera_type"] == "videofreeze":
                # get the experiment (which we have already)
                FreezeThresh = videofreeze_variables_df_avgd.loc[experiment]["FreezeThresh"]
                MinDuration = videofreeze_variables_df_avgd.loc[experiment]["MinDuration"]
            if mouse_row["camera_type"] == "usb":
                pass

            print(mouse_row["animal_id"])
            look_up_list = []
            look_up_list.append(experiment)
            look_up_list.append(vid_file_extension)
            
            for col in general_cols[person]:
                look_up_list.append(mouse_row[col])
            print(look_up_list)

            # find video given a list of strings it should contain
            vid_found = find_file_with_strings(PERSON_ROOT, look_up_list)
            print("vid_found: ", vid_found)
            # get experimental group
            experimental_group = mouse_row["experimental_group"]

            # Get fps of video
            vid_opencv_obj = cv2.VideoCapture(vid_found)
            fps = vid_opencv_obj.get(cv2.CAP_PROP_FPS)
            print("Frames per second:", fps)
            # if look_up_list contains the element 'usb', don't divide by 2,
            # otherwise, divide by 2, i think? yea bc eztrack downsamples by 2 for non-usbs
            if 'usb' in look_up_list:
                fps_eztrack_adjusted = fps
            else:
                fps_eztrack_adjusted = fps / 2 # bc eztrack downsamples by 2
            vid_opencv_obj.release()

            # get correction time in seconds to subtract from total frames
            correction_time_in_frames = math.ceil(time_to_frames(mouse_row[experiment], fps_eztrack_adjusted))
            print("correction_time_in_frames: ", correction_time_in_frames)

            # find calibration video
            chamber = mouse_row["chamber"]

            #to find calibration for extinction_get rid of _1 and _2
            if "_1" in experiment or "_2" in experiment:
                experiment_cali = experiment[:-2]
            else:
                experiment_cali = experiment

            calibration_vid_file_name = find_file_with_strings(CALIBRATION_ROOT, [chamber, experiment_cali])
            print("calibration_vid_file_name: ", calibration_vid_file_name)

            video_dict = {
                'dpath'   : CALIBRATION_ROOT,  
                'file'    : calibration_vid_file_name.split("/")[-1],
                'start'   : calibrate_video_what_frame_to_start, 
                'end'     : None,
                'dsmpl'   : dsmpl,
                'stretch' : dict(width=1, height=1),
                'cal_frms' : number_of_frames_to_calibrate
                }

            img_crp, video_dict = fz.LoadAndCrop(video_dict)
            ####### CALIBRATION #######
            cal_dif_avg, percentile, mt_cutoff = fz.calibrate_custom(video_dict, cal_pix=10000, SIGMA=1)

            ####### FREEZE ANALYSIS #######
            cap = cv2.VideoCapture(vid_found)
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print("length of vid: ", length)

            vid_d_end = length

            video_dict = {
                'dpath'   : PERSON_ROOT,  
                'file'    : vid_found.split("/")[-1],
                'fpath'   : vid_found,
                'start'   : vid_d_start, 
                'end'     : vid_d_end,
                'dsmpl'   : dsmpl,
                'stretch' : dict(width=1, height=1)
            }

            Motion, frames_processed = fz.Measure_Motion(video_dict, mt_cutoff, SIGMA=1)  
            plt_mt = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
                height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
            plt_mt

            Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
            fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)
            print('Average Freezing: {x}%'.format(x=np.average(Freezing)))

            vid_name_no_ext = vid_found.split("/")[-1].split(".")[0]
            freezing_result_filename = f"{vid_name_no_ext}_FreezingOutput.csv"
            freezing_result_path = vid_found.replace(vid_found.split("/")[-1], freezing_result_filename)

            # this processing includes frame correction from when experiment started
            processed_freezing_out_path = freezing_result_path.replace(".csv", f"_frzthr{FreezeThresh}_mindur{MinDuration}_processed.csv")

            df_freezing_out = pd.read_csv(freezing_result_path)
            #finding the timing file for this person and experiment
            timing_filepath = find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment])
            df_timing = ez.timing_file_processing(find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment]), fps_eztrack_adjusted, correction_time_in_frames)
            df_aligned = ez.freezing_alignment(df_freezing_out, df_timing)

            print("processed_freezing_out_path: ", processed_freezing_out_path)
            df_aligned.to_csv(processed_freezing_out_path, index=False)
        

# Plotting

In [33]:
only_pull_columns = ["animal_id", "experimental_group", "camera_type"]
groupby_cols = ["experimental_group", "camera_type"]
camera_type = ["videofreeze", "usb"]
# going through experiment for this person
for experiment in experiment_cols[person]:
    print("experiment: ", experiment)
   
    opsin_group_colors = [mcolors.to_hex((random.random(), random.random(), random.random())), 
                            mcolors.to_hex((random.random(), random.random(), random.random()))]

    timing_filepath = find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment])
    df_timing = pd.read_csv(timing_filepath)
    cs_nums = range(1, len(df_timing) + 1)


    experimental_groups_df = df_start_info.loc[:, only_pull_columns]
    #print(experimental_groups_df.head())
    grouped_df = experimental_groups_df.groupby(groupby_cols).agg(lambda x: list(x))
    d_from_df = grouped_df.T.to_dict(orient='list')

    # flatten 2d array
    for key_1, values in d_from_df.items():
        d_from_df[key_1] = values[0]

    #print("d_from_df: ")
    #print(json.dumps(d_from_df, indent=4))
    
    # need to see what d_from_df prints me out so i can determine data structure can be used to search for
    # a videos directory (experiment->camera_type) and then search for the video file name and see
    # in what opsin group it belongs in
    # is separated by tuples of opsin and camera type
    # got: experiment, camera_type (key), animal_id(value in list)

    # going through each grouping of mice per cam and experiment
    for key_2, values in d_from_df.items():
        camera = key_2[1]
        if camera != "usb":
            print("CURRENT GROUP:", key_2)

            if camera == "videofreeze":
                # get the experiment (which we have already)
                FreezeThresh = videofreeze_variables_df_avgd.loc[experiment]["FreezeThresh"]
                MinDuration = videofreeze_variables_df_avgd.loc[experiment]["MinDuration"]
            if camera == "usb":
                pass

            if person == "Olena":
                curr_dir = find_directory_with_strings([experiment, camera], PERSON_ROOT)
            else:
                curr_dir = find_directory_with_strings([experiment], PERSON_ROOT)

            print(curr_dir)
            #d_from_df is grouped by experimental group and camera type
            # however, we search for vids under the same experiment and camera type for the person
            # which means we willbe able to separate by experimental group

            # new d will be created of avgs of groups
            d_groups = {}   
            num_mice = 0

            
            for file in os.listdir(curr_dir):
                if eztrack_output_processed_suffix in file and ".~" not in file:
                    print("file: ", file)
                    # finding the name of animal embedded in the vid name
                    if person == "Olena":
                        mouse = extract_mouse_id_olena(file)
                    elif person == "Ozge":
                        mouse = extract_mouse_id_ozge(file)
                    elif person == "Patrick":
                        mouse = extract_mouse_id_patrick(file)
                    
                    #print(mouse)
                    experimental_group = None

                    # check if mouse is in one of the opsin groups
                    for key_3, values in d_from_df.items():
                        if mouse in d_from_df[key_3]:
                            experimental_group = key_3[0] 
                            # this will be true for olena & everyone else?
                            
                    #print(mouse, ":", experimental_group)
                    filepath = os.path.join(curr_dir, file)
                    filepath_video = remove_substring(filepath, "_" + eztrack_output_processed_suffix) + ".avi"
                    #print("filepath_video: ", filepath_video)
                    fps_adjusted = get_adjusted_fps(filepath_video, camera)
                    #print("fps_adjusted: ", fps_adjusted)
                    
                    df = pd.read_csv(filepath)
                    frame_lst = list(df["Frame"])

                    timestamps_lst = list(df["Timestamps"])
                    
                    
                    # stamped_lst is the x
                    stamped_lst = ez.overlap_two_lists(frame_lst, timestamps_lst)

                    # modify y to just be binary and not 0 and 100
                    freezing_lst = ez.lst_to_binary_lst(list(df["Freezing"]))
                    #print(freezing_lst)

                    # half_time_window is in seconds
                    x, proportions = ez.bin_data(frame_lst, timestamps_lst,freezing_lst, half_time_window = half_time_window, fps=fps_adjusted, event_tracked=event_tracked)
                    #list_of_freezing_props_all_mice.append(proportions)
                    print("x: ", x)
                    print("proportions: ", proportions)
                    # add to d
                    if experimental_group in d_groups:
                        d_groups[experimental_group].append(proportions)
                    else:
                        d_groups[experimental_group] = []
                        d_groups[experimental_group].append(proportions)

                    num_mice += 1
                #break
            print("d_groups: ")
            print(json.dumps(d_groups, indent=4))
            #del d_groups[None] # delete the None group (idk why it's there)
            keys_to_delete = []

            for key in d_groups:
                if key is None:
                    keys_to_delete.append(key)

            for key in keys_to_delete:
                del d_groups[key]

            csv_data = pd.DataFrame()

            count = 0
            fig, ax = plt.subplots()
            for key_4 in d_groups:
                print("key_4: ", key_4)
                print("key_4 type: ", type(key_4))

                # Convert the list of lists to a NumPy array
                #print("d_groups[key_4]: ",d_groups[key_4])
                array_of_lists = np.array(d_groups[key_4])
                # maybe not necessary
                #array_of_lists = [num for sublist in array_of_lists for num in sublist]
                # Find the length of the longest array
                max_len = max(len(lst) for lst in array_of_lists)

                # Use a list comprehension to create a new list of arrays where the shorter arrays are padded with np.nan
                padded_lists = [np.concatenate([lst, np.full(max_len - len(lst), np.nan)]) for lst in array_of_lists]

                # Now convert the list of arrays to a 2D numpy array
                array_of_lists = np.array(padded_lists)

                print("array_of_lists: ",array_of_lists)

                # Calculate the average of the array along the columns (axis=0)
                average = np.mean(array_of_lists, axis=0)
                print("average")
                print(average)

                # Now you can calculate the standard deviation along the columns
                std_deviation = np.nanstd(array_of_lists, axis=0)
                print("std_deviation")
                print(std_deviation)
                std_error = [std / math.sqrt(num_mice) for std in std_deviation]

                # store stuff in dict
                group_data = pd.DataFrame({
                    'cs_num': cs_nums,
                    'experimental_group': key_4,
                    'average': average,
                    'std_error': std_error
                })
                csv_data = csv_data.append(group_data)

                print("cs_nums len: ", len(cs_nums))
                print("average len: ",len(average))
                print(count)
                print(len(opsin_group_colors))
                if key_4 != None:
                    ax.plot(cs_nums, average, label=key_4, color=opsin_group_colors[count])
                    plt.errorbar(cs_nums, average, yerr = std_error, fmt='-o', color=opsin_group_colors[count], capsize=3)

                count += 1

            outfilename = f"{experiment}_halftimewdw{half_time_window}_frzthr{FreezeThresh}_mindur{MinDuration}_plot.png"
            outpath = "/".join(filepath.split("/")[:-1]) + "/" + outfilename
            print(outpath)

            # After the plot has been saved, save the csv_data DataFrame to a CSV file
            outfilename_csv = f"{experiment}_halftimewdw{half_time_window}_frzthr{FreezeThresh}_mindur{MinDuration}.csv"
            outpath_csv = "/".join(filepath.split("/")[:-1]) + "/" + outfilename_csv
            csv_data.to_csv(outpath_csv, index=False)
            
            ax.set_title(f"Proportion of Freezing - {experiment} (n={num_mice})")
            ax.set_ylabel(f"Proportion")
            ax.set_xlabel(f"CS #")
            plt.legend()
            fig.savefig(outpath)
            plt.close()
        
        #break

experiment:  conditioning
['olena', 'fc_info', 'conditioning']
CURRENT GROUP: ('AS-Gi', 'videofreeze')
/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080822_conditioning/VideoFreeze
file:  hSyn-AS-Gi-12_FreezingOutput_processed.csv
x:  ['CS ON:0:03:00 : 5400', 'CS ON:0:04:30 : 8100', 'CS ON:0:06:30 : 11700']
proportions:  [0.0, 0.0, 0.0]
file:  hSyn-AS-Gi-10_FreezingOutput_processed.csv
x:  ['CS ON:0:03:00 : 5400', 'CS ON:0:04:30 : 8100', 'CS ON:0:06:30 : 11700']
proportions:  [0.0, 0.0, 0.0]
file:  hSyn-AS-Gi-11_FreezingOutput_processed.csv
x:  ['CS ON:0:03:00 : 5400', 'CS ON:0:04:30 : 8100', 'CS ON:0:06:30 : 11700']
proportions:  [0.0, 0.0, 0.0]
file:  hSyn-AS-Gi-4_FreezingOutput_processed.csv
x:  ['CS ON:0:03:00 : 5400', 'CS ON:0:04:30 : 8100', 'CS ON:0:06:30 : 11700']
proportions:  [0.0, 0.0, 0.0]
file:  hSyn-AS-Gi-5_FreezingOutput_processed.csv
x:  ['CS ON:0:03:00 : 5400', 'CS ON:0:04:30 : 8100', 'CS ON:0:06:30 : 11700']
proportions:  [0.0, 0.0, 0.5466666666666666]
f